In [1]:
import ultralytics
from ultralytics import YOLO
import os
import cv2
import time

# Set CUDA Environment Variables
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
# o

CUDA_VERSION = '12.1'
os.environ['PATH'] = '/usr/local/cuda-' + CUDA_VERSION + '/bin:' + os.environ.get('PATH', '')
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-' + CUDA_VERSION + '/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')


# Change these parameters to fit your needs
EPOCHS = 1
IMG_SIZE = 640

# Whether or not to use ray tune for hyperparameter sweep / tuning
USE_RAY_TUNE = False
# Number of iterations for hyperparameter sweep / tuning
TUNE_ITERS = 1

CURR_DIR = os.getcwd()
WORKSPACE_DIR = os.path.dirname(CURR_DIR)
DATASETS_DIR = WORKSPACE_DIR + '/datasets/'

# Flag to resume training from a previous checkpoint (false if training from scratch)
RESUME_TRAINING = False

# Path to trained model weights
MODELS_PATH = WORKSPACE_DIR + 'models/'
MODEL_PATH = MODELS_PATH + 'yolov8_fine_tune.pt'

# This line prevents the Kernel from crashing when running model.train() which calls a plotting function
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Check System Information

In [ ]:
ultralytics.utils.checks.collect_system_info()

# Train Model

In [ ]:
# Load yolov8 nano segmentation model
if RESUME_TRAINING:
    # TODO: Change this to the path of the model you want to resume training from
    model = YOLO('yolov8n-seg.pt', resume=True)
# Load yolov8 nano segmentation model
else:
    model = YOLO('yolov8n-seg.pt')

# Find dataset images
CURR_DATASET = DATASETS_DIR + '/YOLOv8_finetune.v1i.yolov8-car-label' #TODO: Change this to the dataset you want to train on
TRAIN_DATA = CURR_DATASET + '/data.yaml'
TEST_PATH = CURR_DATASET + '/test/images/'
SAVE_PATH = CURR_DATASET + '/test/annotation_results'
# Make sure the save path exists
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

start_time = time.time()
# By default, the model trains on a single GPU
model.train(data=TRAIN_DATA, epochs=EPOCHS, imgsz=IMG_SIZE)
end_time = time.time()
training_time = end_time - start_time
print("Time to train: ", training_time)


# Hyperparameter Tuning

In [ ]:
# Runs a hyperparameter sweep and selects the best hyperparameters
model.tune(use_ray=USE_RAY_TUNE, iterations=TUNE_ITERS)

# Test Model

In [ ]:
# Inference fine-tuned model on test images and save results
for file in os.listdir(TEST_PATH):
    file_path = os.path.join(TEST_PATH, file)
    output = model.predict(file_path)
    save_path = os.path.join(SAVE_PATH, file)
    cv2.imwrite(save_path, output[0].plot())

print("Inference on test set complete. Results saved to: ", SAVE_PATH)

# Save Weights and Export Model

In [ ]:
model.save(MODEL_PATH) # Save the model as a .pt file
model.export(save_dir=MODELS_PATH, format='onnx', half=True)